In [1]:
import numpy as np
import pandas as pd
import datetime as dt

### Source DataFrame

In [2]:
# when is the next train coming?
df_schedule = pd.read_csv('train_schedules.csv')
df_schedule.head(2)

,5th Avenue,Abad Santos,Baclaran,Balintawak,Bambang,Blumentritt,Carriedo,Central Terminal,Doroteo Jose,EDSA,Gil Puyat,Libertad,Monumento,Pedro Gil,Quirino,R Papa,Roosevelt,Tayuman,United Nations,Vito Cruz
0,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600,1531686600
1,1531686783,1531686781,1531686782,1531686784,1531686783,1531686782,1531686786,1531686783,1531686784,1531686786,1531686784,1531686785,1531686784,1531686781,1531686785,1531686784,1531686780,1531686786,1531686782,1531686785


In [3]:
# distances between stations
df_distances = pd.read_csv('df_distances.csv')
df_distances.head(3)

,start_station,end_station,distance,travel_time
0,Baclaran,EDSA,0.588,1.764
1,Baclaran,Libertad,1.598,4.794
2,EDSA,Libertad,1.010,3.030


In [4]:
# number of passengers waiting per minute per station
df_queue = pd.read_csv('df_passengers_per_min_per_station.csv')
df_queue.head(3)

,station,min_count,card_num,Timestamp
0,5th Avenue,25528111,9,1531686660
1,5th Avenue,25528112,13,1531686720
2,5th Avenue,25528113,5,1531686780


In [5]:
get_time = lambda x: dt.datetime.fromtimestamp(x).time()

In [6]:
# check load factor: light, moderate, heavy
def check_load_factor(time, is_raining=False, is_weekday=True):
    h = int(dt.datetime.fromtimestamp(time//1000).strftime('%H'))
    if (h >= 11 and h < 15) or (h >= 21):
        return 'lightly'
    elif (h < 7) or (h >= 9 and h < 11) or (h >= 15 and h < 16)\
        or (h >= 19 and h < 21):
        return 'moderately'
    else:
        return 'heavy'

In [7]:
def query(time, station1, station2):
    scheds =  df_schedule.loc[df_schedule.loc[:, station1].apply(get_time)
                              >= get_time(time//1000), station1]
    
    scheds = scheds.apply(lambda x: dt.datetime.fromtimestamp(x)\
                                      .strftime('%I:%M%p')).head().values
    
    travel_time = df_distances.loc[(df_distances.start_station==station1)
                                   & (df_distances.end_station==station2),
                                   'travel_time'].values[0]
    
    queue_people = df_queue.loc[(df_queue.station==station1) &
                                (df_queue.Timestamp.apply(get_time)
                                 >= get_time(time//1000))]\
                            .sort_values('Timestamp').head(15).card_num.sum()
    
    
    queue = int((queue_people / 150) * 3 // 1)
    
    load_factor = check_load_factor(time)
    
    msg = 'Arrival schedules of the next '
    msg += f'five trains in {station1} station: '
    msg += ', '.join(scheds[:-1])
    msg += f', and {scheds[-1]}. '
    msg += f'Expected waiting time from station is {queue} minutes. '
    msg += f'Incoming trains are expected to be {load_factor} loaded. '
    msg += f'Total travel time from {station1} to {station2} '
    msg += f'is {int(np.round(travel_time))} minutes. '
    msg += 'Ingat po sa byahe!'
    
    return msg.split('. ')

In [8]:
query(1531576038314, 'Baclaran', 'Monumento')

['Arrival schedules of the next five trains in Baclaran station: 09:49PM, 09:52PM, 09:55PM, 09:58PM, and 10:01PM',
 'Expected waiting time from station is 6 minutes',
 'Incoming trains are expected to be lightly loaded',
 'Total travel time from Baclaran to Monumento is 42 minutes',
 'Ingat po sa byahe!']